In [31]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [32]:
#Load csv files 
HourlyMCP = pd.read_csv('../Data/ModelData/HourlyMCPDemo.csv', delimiter=',')
Fuel_Cost_Forecast_Normalized = pd.read_csv('../Data/ModelData/FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Asset_Data_New = pd.read_csv('../Data/ModelData/Generators_AssetData_New.csv', delimiter=',')

In [33]:
#show the first 5 rows of the data
HourlyMCP.head()
#Fuel_Cost_Forecast_Normalized.head()
#Generation_Asset_Data_New.head()

,Year,Season,Hour,MCP
0,1,1,1,75
1,1,1,2,75
2,1,1,3,75
3,1,1,4,75
4,1,1,5,75


In [34]:
#Define modelling time horizon
#Amount of years
Y = 1

#Amount of seasons in a year
S = 1

#Amount of hours in a season
H = 24

In [ ]:
#Initialize the model
m = gp.Model("InvestmentModel")

#Define the decision variables
#Capacity Investment in invetment opportunity i defined by Generation_Asset_Data_New['Unit #']
Inv_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], vtype=GRB.CONTINUOUS, name="Inv_NewCap")
#Energy generated by investment opportunity i in hour h
P_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], H, vtype=GRB.CONTINUOUS, name="P_NewCap")

#Extract the values from the pandas Series
HourlyMCP_values = HourlyMCP['MCP'].values
CapInv_values = Generation_Asset_Data_New['C_CapInv ($/MW)'].values

#Define the objective function
#Objective function is to maximize the profit
m.setObjective(gp.quicksum(P_NewCap[i,h]*HourlyMCP_values[h] for i in Generation_Asset_Data_New['Unit #'] for h in range(H)) - gp.quicksum(Inv_NewCap[i]*CapInv_values[j] for j, i in enumerate(Generation_Asset_Data_New['Unit #'])), GRB.MAXIMIZE)